In [19]:
from bs4 import BeautifulSoup
from datetime import datetime
import time

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

In [ ]:
os.chdir("/Users/avascharfstein/Desktop/Dartmouth/mds-final")

In [20]:
import requests as rq

In [21]:
date = '2023-01-02'
url = f'https://www.wtatennis.com/rankings/singles'
op = webdriver.ChromeOptions()
op.add_argument('headless') # don't open up the new chrome page
driver = webdriver.Chrome(options=op)
driver.get(url)

# filter_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'rankings-filter__content')))
# filter_element.click()

# date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'js-datepicker-trigger')))

# Click on the filter to open the date picker
# wait = WebDriverWait(driver, 10)
# filter_element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'rankings-filter__content js-filter-trigger js-filter-disable')))
# filter_element.click()

# # Wait for the date picker to appear (you might need to adjust the sleep time)
# time.sleep(2)

# # Select the date input field and clear the existing date
# date_input = driver.find_element_by_css_selector('.js-datepicker-trigger')
# date_input.clear()

# # Type the new date into the input field
# date_input.send_keys(date)

# # Press Enter to apply the new date
# date_input.send_keys('\n')

# # Wait for the dynamic content to load (you might need to adjust the sleep time)
# time.sleep(5)

# # Get the page source after JavaScript has executed
# page_source = driver.page_source

TimeoutException: Message: 


In [36]:
from selenium.webdriver.common.action_chains import ActionChains

reset_button = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'reset-button'))
    )

driver.execute_script("arguments[0].click();", reset_button)


print(driver.find_elements(By.CLASS_NAME, 'rankings-filter__content js-filter-trigger js-filter-disable'))


[]


In [11]:
soup = BeautifulSoup(page_source, 'html.parser')
rows = soup.body.find('tbody',class_='js-rankings-body').findAll('tr',class_='rankings__row')
rows

[<tr class="rankings__row"> <td class="rankings__cell"> <span class="rankings__rank">1</span> <span class="rankings__movement rankings__movement--none"> - </span> <a aria-label="Iga Swiatek" class="rankings__player-link rankings__player-link--chrome" href="//www.wtatennis.com/players/326408/iga-swiatek" title="Iga Swiatek">Iga Swiatek</a> </td> <td class="rankings__cell rankings__cell--player"> <div class="player-headshot player-headshot--closeup rankings__player-picture u-observed head-only head-only--large" data-init-when-visible="true" data-initialised="true" data-object-fit="true" data-observable-id="player-headshot_1707247617863_0.9357821878420516" data-picture-in-view="false" data-player="326408" data-size="RANKINGS" data-widget="player-headshot"> <picture class="js-picture-element player-headshot__photo object-fit-cover-picture" data-photo-id="" data-size-modifiers="head-only,head-only--large"> <source media="(max-width: 840px)" srcset="https://photoresources.wtatennis.com/photo

In [7]:

def fetch_rankings(date):
    url = 'https://www.wtatennis.com/rankings/singles'
    
    # Construct the payload with the date parameters
    payload = {
        'data-date': date,
    }

    response = rq.post(url, data=payload)

    print(response)

    if response.status_code == 200:
        content_type = response.headers.get('Content-Type')
        print(f"Content-Type: {content_type}")

        # Process the response based on its content type
        if 'json' in content_type.lower():
            rankings_data = response.json()
            # Process JSON data
            print(rankings_data)
        elif 'html' in content_type.lower():
            rankings_html = response.text
            # Process HTML data
            print(rankings_html)
        else:
            print("Unknown content type. Handle accordingly.")
    else:
        print('Failed to fetch rankings')

# Example usage: Fetch rankings for different date ranges
start_date = '2023-12-01'  # Replace with your desired start date

rankings_data = fetch_rankings(start_date)
if rankings_data:
    # Process and use the retrieved rankings data as needed
    print(rankings_data)

<Response [403]>
Failed to fetch rankings


PDF to csv?

In [9]:
#pip install tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 6.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tabula

In [14]:
tabula.convert_into("data/practice_data_singles.pdf", "data/practice_data.csv", pages= "all", output_format= "csv")

In [17]:
import pandas as pd
df = pd.read_csv("data/practice_data.csv")

ParserError: Error tokenizing data. C error: Expected 11 fields in line 205, saw 12


Trying a new website for womens ranking extraction

In [32]:
tourney_date_dict = {'2023-06-26': '2023-06-26', '2023-05-22': '2023-05-22', '2024-01-01': '2024-01-07', '2023-08-21': '2023-08-22'}

In [44]:
wta_ranks = []
for week in list(tourney_date_dict.keys()):
    player_ranks = {}
    url = "https://www.tennisexplorer.com/ranking/wta-women/?date="+week
    for p in range(1,29):
        url_p = url+"&page="+str(p)
        page = rq.get(url_p)
        content = page.content
        soup = BeautifulSoup(content)
        rows = soup.body.find('tbody', class_='flags').findAll('tr')
        for row in rows:
            rank = row.find('td', class_ = 'rank first').text.strip()
            rank_numeric = float(rank.replace('.', ''))
            name = row.find('td', class_ = 't-name').text.strip()
            #country = row.find('td', class_="tl").a.text.strip()
            player_ranks[name] = rank_numeric

    week_ranks = pd.DataFrame(list(player_ranks.items()), columns = ['player','rank'])
    week_ranks['week'] = tourney_date_dict[week]
    print(week,"(",tourney_date_dict[week], ")","ATP rankings extracted")

    wta_ranks.append(week_ranks)


2023-06-26 ( 2023-06-26 ) ATP rankings extracted
2023-05-22 ( 2023-05-22 ) ATP rankings extracted
2024-01-01 ( 2024-01-07 ) ATP rankings extracted
2023-08-21 ( 2023-08-22 ) ATP rankings extracted


In [46]:
wta_ranks = pd.concat(wta_ranks)
wta_ranks

,player,rank,week
0,Swiatek Iga,1.0,2023-06-26
1,Sabalenka Aryna,2.0,2023-06-26
2,Rybakina Elena,3.0,2023-06-26
3,Pegula Jessica,4.0,2023-06-26
4,Garcia Caroline,5.0,2023-06-26
...,...,...,...
1395,Faltinger Ines,1390.0,2023-08-22
1396,Goldsmith Weinreich Paloma,1390.0,2023-08-22
1397,Ionescu Iulia Andreea,1390.0,2023-08-22
1398,Joint Maya,1390.0,2023-08-22


In [52]:
wta_ranks.to_csv("data/wta_ranks.csv", index=False)